## Process Control Framework - Main Notebook
#### This notebook creates a workflow for a given job based on its child job configuration

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("esp_job_id", "", "ESP Job ID")

In [0]:
esp_job_id = dbutils.widgets.get("esp_job_id")

In [0]:
print(f"esp_job_id: {esp_job_id}")

In [0]:
from src.metadata_utils import get_job_details
from src.workflow_utils import create_workflow_yaml

In [0]:
from pathlib import Path
import os

def get_project_root(marker="databricks.yml"):
  current_file = Path(os.path.abspath("")).resolve()
  for parent in current_file.parents:
    if (parent / marker).exists():
      return parent
  raise Exception(f"Could not find project root with marker {marker}")

project_root_path = get_project_root()
print(f"Project root path: {project_root_path}")

In [0]:
try:
    print(f"Starting process control for esp_job_id = {esp_job_id}")

    # Step 1: Get child jobs
    parent_info, child_jobs = get_job_details(spark, esp_job_id)

    if not parent_info or not child_jobs:
        raise Exception(f"Could not find job details for ESP job id : {esp_job_id}")
    
    print(f"Parent job info: {parent_info}")
    print(f"Child jobs: {child_jobs}")
    print(f"Found {len(child_jobs)} child jobs for {esp_job_id}")
    
    # Step 2: Build workflow tasks

    job_yaml_path = create_workflow_yaml(esp_job_id, parent_info, child_jobs, project_root_path)

    print(f"YAML file created at: {job_yaml_path}")

    if job_yaml_path:
        print(f"SUCCESS: Created workflow yaml")
    else:
        raise Exception("FAILED: Could not create workflow yaml")
    
except Exception as e:
    error_msg = f"Error in process control: {str(e)}"
    print(error_msg)
    raise Exception(f"FAILED: {error_msg}")
